# 🔄 反向传播算法的数学推导

## 🎯 学习目标

反向传播（Backpropagation）是训练神经网络的核心算法。本教程将从数学层面深入剖析这个算法。

**学完本教程你将能够：**

1. ✅ 完整推导单层网络的反向传播公式
2. ✅ 理解多层网络中误差的逐层反向传播机制
3. ✅ 掌握链式法则在深度网络中的应用
4. ✅ 理解梯度消失/爆炸的数学根源
5. ✅ 从零实现多层网络的完整前向和反向传播
6. ✅ 使用自动微分验证手动推导的正确性

---

## 📚 先修知识

- 矩阵微分与梯度计算 → `05_matrix_calculus.ipynb`
- 链式法则 → `02_calculus.ipynb`

---

## 📖 目录

1. **单层网络的反向传播**
   - 前向传播：$\mathbf{z} = W\mathbf{x} + \mathbf{b}$, $\mathbf{a} = \sigma(\mathbf{z})$
   - 损失函数：$L = \text{loss}(\mathbf{a}, \mathbf{y})$
   - 反向传播推导：$\frac{\partial L}{\partial W}$, $\frac{\partial L}{\partial \mathbf{b}}$, $\frac{\partial L}{\partial \mathbf{x}}$
   - 常见激活函数的导数

2. **多层网络的链式法则**
   - 两层网络的完整推导
   - 误差的反向传播机制
   - 梯度消失/爆炸的数学根源
   - 残差连接的数学解释

3. **向量化实现**
   - 批量数据的反向传播
   - 广播机制的应用
   - 从零实现多层网络
   - 与自动微分对比验证

**预计学习时间：** 4-5 小时

---

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

# 设置打印精度
np.set_printoptions(precision=4, suppress=True)

print("✅ 环境配置完成！")
print(f"NumPy 版本: {np.__version__}")

---

# 1️⃣ 单层网络的反向传播

## 1.1 网络结构定义

考虑最简单的单层神经网络：

$$
\begin{aligned}
\mathbf{z} &= W\mathbf{x} + \mathbf{b} \quad &\text{(线性变换)} \\
\mathbf{a} &= \sigma(\mathbf{z}) \quad &\text{(激活函数)} \\
L &= \text{Loss}(\mathbf{a}, \mathbf{y}) \quad &\text{(损失函数)}
\end{aligned}
$$

**形状约定：**
- $\mathbf{x} \in \mathbb{R}^{n}$ - 输入向量（$n$ 个特征）
- $W \in \mathbb{R}^{m \times n}$ - 权重矩阵
- $\mathbf{b} \in \mathbb{R}^{m}$ - 偏置向量
- $\mathbf{z} \in \mathbb{R}^{m}$ - 线性输出（激活前）
- $\mathbf{a} \in \mathbb{R}^{m}$ - 激活输出（激活后）
- $\mathbf{y} \in \mathbb{R}^{m}$ - 目标输出
- $L \in \mathbb{R}$ - 损失（标量）

---

## 1.2 计算图可视化

```
输入       线性层          激活函数        损失函数
 x  ──→  [W, b]  ──→  z  ──→  σ  ──→  a  ──→  Loss  ──→  L
                                               ↑
                                               y (目标)
```

**反向传播方向：**

```
∂L/∂x ←─ ∂L/∂W ←─ ∂L/∂z ←─ ∂L/∂a ←─ ∂L/∂L = 1
         ∂L/∂b ←─┘
```

---

## 1.3 前向传播：逐步计算

### 🔢 数值示例

设定具体数值来帮助理解：

In [ ]:
# 定义激活函数和导数

def sigmoid(z):
    """
    Sigmoid 激活函数: σ(z) = 1 / (1 + exp(-z))
    
    性质:
    - 输出范围: (0, 1)
    - 可微分
    - 饱和性: z很大或很小时导数接近0
    """
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    """
    Sigmoid 导数: σ'(z) = σ(z) * (1 - σ(z))
    
    推导:
    σ(z) = 1 / (1 + e^(-z))
    
    令 u = 1 + e^(-z), 则 σ = 1/u
    dσ/dz = -1/u² * du/dz
          = -1/u² * (-e^(-z))
          = e^(-z) / (1 + e^(-z))²
    
    化简:
    = [1 / (1 + e^(-z))] * [e^(-z) / (1 + e^(-z))]
    = σ(z) * [1 - σ(z)]  ← 常用形式
    """
    s = sigmoid(z)
    return s * (1 - s)

# 前向传播示例
print("=" * 60)
print("单层网络前向传播示例")
print("=" * 60)

# 定义参数
n = 3  # 输入维度
m = 2  # 输出维度

x = np.array([1.0, 2.0, 3.0])           # (3,) 输入
W = np.array([[0.1, 0.2, 0.3],          # (2, 3) 权重
              [0.4, 0.5, 0.6]])
b = np.array([0.1, 0.2])                # (2,) 偏置
y = np.array([1.0, 0.0])                # (2,) 目标

print(f"\n输入 x: {x}")
print(f"权重 W:\n{W}")
print(f"偏置 b: {b}")
print(f"目标 y: {y}")

# Step 1: 线性变换 z = Wx + b
z = np.dot(W, x) + b
print(f"\n[前向] z = Wx + b = {z}")
print(f"  计算: [{W[0] @ x + b[0]:.4f}, {W[1] @ x + b[1]:.4f}]")

# Step 2: 激活函数 a = σ(z)
a = sigmoid(z)
print(f"\n[前向] a = σ(z) = {a}")

# Step 3: 损失函数（使用 MSE）
# L = 0.5 * ||a - y||²
loss = 0.5 * np.sum((a - y) ** 2)
print(f"\n[前向] L = 0.5 * ||a - y||² = {loss:.6f}")
print(f"  误差 (a - y) = {a - y}")

---

## 1.4 反向传播：完整推导

### 🎯 目标

计算损失函数对所有参数的梯度：

$$
\frac{\partial L}{\partial W}, \quad \frac{\partial L}{\partial \mathbf{b}}, \quad \frac{\partial L}{\partial \mathbf{x}}
$$

---

### 📝 推导步骤

#### Step 1: $\frac{\partial L}{\partial \mathbf{a}}$ - 损失对激活值的梯度

使用 MSE 损失：$L = \frac{1}{2} \sum_{i=1}^{m} (a_i - y_i)^2$

$$
\frac{\partial L}{\partial a_i} = a_i - y_i \quad \Rightarrow \quad \frac{\partial L}{\partial \mathbf{a}} = \mathbf{a} - \mathbf{y}
$$

---

#### Step 2: $\frac{\partial L}{\partial \mathbf{z}}$ - 损失对线性输出的梯度

使用链式法则：

$$
\frac{\partial L}{\partial \mathbf{z}} = \frac{\partial L}{\partial \mathbf{a}} \frac{\partial \mathbf{a}}{\partial \mathbf{z}}
$$

因为 $\mathbf{a} = \sigma(\mathbf{z})$ 是逐元素操作，Jacobian 是对角矩阵：

$$
\frac{\partial \mathbf{a}}{\partial \mathbf{z}} = \text{diag}(\sigma'(z_1), \sigma'(z_2), \ldots, \sigma'(z_m))
$$

矩阵-向量乘法退化为**逐元素乘法**（Hadamard product $\odot$）：

$$
\boxed{\frac{\partial L}{\partial \mathbf{z}} = (\mathbf{a} - \mathbf{y}) \odot \sigma'(\mathbf{z}) = \delta}
$$

我们将 $\delta$ 称为**误差向量**。

---

#### Step 3: $\frac{\partial L}{\partial W}$ - 损失对权重的梯度

$$
\frac{\partial L}{\partial W} = \frac{\partial L}{\partial \mathbf{z}} \frac{\partial \mathbf{z}}{\partial W}
$$

因为 $z_i = \sum_{j=1}^{n} W_{ij} x_j + b_i$，所以：

$$
\frac{\partial z_i}{\partial W_{ij}} = x_j
$$

因此：

$$
\frac{\partial L}{\partial W_{ij}} = \delta_i \cdot x_j
$$

矩阵形式：

$$
\boxed{\frac{\partial L}{\partial W} = \delta \mathbf{x}^T}
$$

**形状验证：**
- $\delta \in \mathbb{R}^{m}$ (列向量)
- $\mathbf{x}^T \in \mathbb{R}^{1 \times n}$ (行向量)
- $\delta \mathbf{x}^T \in \mathbb{R}^{m \times n}$ ✅ 与 $W$ 形状一致

---

#### Step 4: $\frac{\partial L}{\partial \mathbf{b}}$ - 损失对偏置的梯度

$$
\frac{\partial L}{\partial \mathbf{b}} = \frac{\partial L}{\partial \mathbf{z}} \frac{\partial \mathbf{z}}{\partial \mathbf{b}}
$$

因为 $\mathbf{z} = W\mathbf{x} + \mathbf{b}$，所以 $\frac{\partial \mathbf{z}}{\partial \mathbf{b}} = I$ (单位矩阵)。

$$
\boxed{\frac{\partial L}{\partial \mathbf{b}} = \delta}
$$

---

#### Step 5: $\frac{\partial L}{\partial \mathbf{x}}$ - 损失对输入的梯度（可选）

如果要将梯度继续反向传播到上一层，需要计算：

$$
\frac{\partial L}{\partial \mathbf{x}} = \frac{\partial L}{\partial \mathbf{z}} \frac{\partial \mathbf{z}}{\partial \mathbf{x}}
$$

因为 $\mathbf{z} = W\mathbf{x} + \mathbf{b}$，所以 $\frac{\partial \mathbf{z}}{\partial \mathbf{x}} = W$。

$$
\boxed{\frac{\partial L}{\partial \mathbf{x}} = W^T \delta}
$$

**形状验证：**
- $W^T \in \mathbb{R}^{n \times m}$
- $\delta \in \mathbb{R}^{m}$
- $W^T \delta \in \mathbb{R}^{n}$ ✅ 与 $\mathbf{x}$ 形状一致

---

In [ ]:
# 反向传播计算（使用前面的数据）

print("=" * 60)
print("单层网络反向传播计算")
print("=" * 60)

# Step 1: ∂L/∂a = a - y
dL_da = a - y
print(f"\n[反向] ∂L/∂a = a - y = {dL_da}")

# Step 2: ∂L/∂z = ∂L/∂a ⊙ σ'(z)
dL_dz = dL_da * sigmoid_derivative(z)  # 逐元素乘法
print(f"\n[反向] ∂L/∂z = ∂L/∂a ⊙ σ'(z)")
print(f"  σ'(z) = {sigmoid_derivative(z)}")
print(f"  δ = ∂L/∂z = {dL_dz}")

# Step 3: ∂L/∂W = δ x^T
dL_dW = np.outer(dL_dz, x)  # 外积
print(f"\n[反向] ∂L/∂W = δ x^T")
print(f"  形状: ({dL_dz.shape[0]},) ⊗ ({x.shape[0]},) = ({dL_dW.shape[0]}, {dL_dW.shape[1]})")
print(f"  ∂L/∂W =\n{dL_dW}")

# Step 4: ∂L/∂b = δ
dL_db = dL_dz
print(f"\n[反向] ∂L/∂b = δ = {dL_db}")

# Step 5: ∂L/∂x = W^T δ (可选)
dL_dx = np.dot(W.T, dL_dz)
print(f"\n[反向] ∂L/∂x = W^T δ")
print(f"  形状: ({W.T.shape[0]}, {W.T.shape[1]}) @ ({dL_dz.shape[0]},) = ({dL_dx.shape[0]},)")
print(f"  ∂L/∂x = {dL_dx}")

print("\n" + "="*60)
print("梯度形状总结")
print("="*60)
print(f"参数 W: {W.shape} -> 梯度 ∂L/∂W: {dL_dW.shape} ✅")
print(f"参数 b: {b.shape} -> 梯度 ∂L/∂b: {dL_db.shape} ✅")
print(f"输入 x: {x.shape} -> 梯度 ∂L/∂x: {dL_dx.shape} ✅")

---

## 1.5 常见激活函数的导数

不同的激活函数有不同的导数，这直接影响梯度的计算。

| 激活函数 | 公式 $\sigma(z)$ | 导数 $\sigma'(z)$ | 特点 |
|---------|------------------|-------------------|------|
| **Sigmoid** | $\frac{1}{1+e^{-z}}$ | $\sigma(z)(1-\sigma(z))$ | 饱和，梯度消失 |
| **Tanh** | $\frac{e^z - e^{-z}}{e^z + e^{-z}}$ | $1 - \tanh^2(z)$ | 零中心，饱和 |
| **ReLU** | $\max(0, z)$ | $\begin{cases} 1 & z > 0 \\ 0 & z \leq 0 \end{cases}$ | 不饱和，Dead ReLU |
| **Leaky ReLU** | $\max(0.01z, z)$ | $\begin{cases} 1 & z > 0 \\ 0.01 & z \leq 0 \end{cases}$ | 缓解 Dead ReLU |

---

In [ ]:
# 实现常见激活函数及其导数

def tanh(z):
    """双曲正切函数"""
    return np.tanh(z)

def tanh_derivative(z):
    """
    Tanh 导数: tanh'(z) = 1 - tanh²(z)
    
    推导:
    tanh(z) = (e^z - e^(-z)) / (e^z + e^(-z))
    
    令 t = tanh(z)
    dt/dz = 1 - t²
    """
    return 1 - np.tanh(z) ** 2

def relu(z):
    """ReLU: Rectified Linear Unit"""
    return np.maximum(0, z)

def relu_derivative(z):
    """
    ReLU 导数: 
    - z > 0: 导数 = 1
    - z <= 0: 导数 = 0
    
    注意: z=0 处不可导，通常定义为 0
    """
    return (z > 0).astype(float)

def leaky_relu(z, alpha=0.01):
    """Leaky ReLU: 允许负值有小梯度"""
    return np.where(z > 0, z, alpha * z)

def leaky_relu_derivative(z, alpha=0.01):
    """
    Leaky ReLU 导数:
    - z > 0: 导数 = 1
    - z <= 0: 导数 = alpha (通常 0.01)
    """
    return np.where(z > 0, 1.0, alpha)

# 可视化激活函数及其导数
z_range = np.linspace(-5, 5, 200)

fig, axes = plt.subplots(2, 4, figsize=(20, 10))

# 定义激活函数列表
activations = [
    ("Sigmoid", sigmoid, sigmoid_derivative),
    ("Tanh", tanh, tanh_derivative),
    ("ReLU", relu, relu_derivative),
    ("Leaky ReLU", leaky_relu, leaky_relu_derivative)
]

for i, (name, func, deriv) in enumerate(activations):
    # 上排：激活函数
    axes[0, i].plot(z_range, func(z_range), linewidth=2.5, color='blue')
    axes[0, i].axhline(y=0, color='k', linewidth=0.5)
    axes[0, i].axvline(x=0, color='k', linewidth=0.5)
    axes[0, i].set_title(f"{name} 激活函数", fontsize=14, fontweight='bold')
    axes[0, i].set_xlabel('z', fontsize=12)
    axes[0, i].set_ylabel('σ(z)', fontsize=12)
    axes[0, i].grid(True, alpha=0.3)
    
    # 下排：导数
    axes[1, i].plot(z_range, deriv(z_range), linewidth=2.5, color='red')
    axes[1, i].axhline(y=0, color='k', linewidth=0.5)
    axes[1, i].axvline(x=0, color='k', linewidth=0.5)
    axes[1, i].set_title(f"{name} 导数", fontsize=14, fontweight='bold')
    axes[1, i].set_xlabel('z', fontsize=12)
    axes[1, i].set_ylabel("σ'(z)", fontsize=12)
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n观察要点:")
print("1. Sigmoid/Tanh: 在 |z| 很大时导数接近 0 → 梯度消失")
print("2. ReLU: z < 0 时导数为 0 → Dead ReLU 问题")
print("3. Leaky ReLU: z < 0 时保留小梯度 → 缓解 Dead ReLU")

---

# 2️⃣ 多层网络的链式法则

## 2.1 两层网络结构

$$
\begin{aligned}
\mathbf{z}^{(1)} &= W^{(1)} \mathbf{x} + \mathbf{b}^{(1)} \quad &\text{(第1层线性)} \\
\mathbf{a}^{(1)} &= \sigma(\mathbf{z}^{(1)}) \quad &\text{(第1层激活)} \\
\mathbf{z}^{(2)} &= W^{(2)} \mathbf{a}^{(1)} + \mathbf{b}^{(2)} \quad &\text{(第2层线性)} \\
\mathbf{a}^{(2)} &= \sigma(\mathbf{z}^{(2)}) \quad &\text{(第2层激活)} \\
L &= \frac{1}{2} \|\mathbf{a}^{(2)} - \mathbf{y}\|^2 \quad &\text{(MSE损失)}
\end{aligned}
$$

**形状约定：**
- $\mathbf{x} \in \mathbb{R}^{n_0}$ - 输入
- $\mathbf{a}^{(1)} \in \mathbb{R}^{n_1}$ - 隐藏层
- $\mathbf{a}^{(2)} \in \mathbb{R}^{n_2}$ - 输出层
- $W^{(1)} \in \mathbb{R}^{n_1 \times n_0}$, $W^{(2)} \in \mathbb{R}^{n_2 \times n_1}$

---

## 2.2 反向传播推导

### 🔙 从输出层向输入层反向传播

#### 输出层（第2层）

$$
\delta^{(2)} = \frac{\partial L}{\partial \mathbf{z}^{(2)}} = (\mathbf{a}^{(2)} - \mathbf{y}) \odot \sigma'(\mathbf{z}^{(2)})
$$

$$
\frac{\partial L}{\partial W^{(2)}} = \delta^{(2)} (\mathbf{a}^{(1)})^T
$$

$$
\frac{\partial L}{\partial \mathbf{b}^{(2)}} = \delta^{(2)}
$$

#### 隐藏层（第1层）- 关键步骤！

$$
\frac{\partial L}{\partial \mathbf{z}^{(1)}} = \frac{\partial L}{\partial \mathbf{a}^{(1)}} \frac{\partial \mathbf{a}^{(1)}}{\partial \mathbf{z}^{(1)}}
$$

**Step 1:** 计算 $\frac{\partial L}{\partial \mathbf{a}^{(1)}}$

由于 $\mathbf{z}^{(2)} = W^{(2)} \mathbf{a}^{(1)} + \mathbf{b}^{(2)}$，使用链式法则：

$$
\frac{\partial L}{\partial \mathbf{a}^{(1)}} = \frac{\partial L}{\partial \mathbf{z}^{(2)}} \frac{\partial \mathbf{z}^{(2)}}{\partial \mathbf{a}^{(1)}} = (W^{(2)})^T \delta^{(2)}
$$

**Step 2:** 乘以激活函数导数

$$
\boxed{\delta^{(1)} = \left( (W^{(2)})^T \delta^{(2)} \right) \odot \sigma'(\mathbf{z}^{(1)})}
$$

**Step 3:** 计算梯度

$$
\frac{\partial L}{\partial W^{(1)}} = \delta^{(1)} \mathbf{x}^T
$$

$$
\frac{\partial L}{\partial \mathbf{b}^{(1)}} = \delta^{(1)}
$$

---

### 🔍 关键洞察

误差反向传播的通用公式：

$$
\delta^{(l)} = \left( (W^{(l+1)})^T \delta^{(l+1)} \right) \odot \sigma'(\mathbf{z}^{(l)})
$$

- $(W^{(l+1)})^T \delta^{(l+1)}$: 将误差从第 $l+1$ 层传播到第 $l$ 层
- $\odot \sigma'(\mathbf{z}^{(l)})$: 乘以激活函数导数（局部梯度）

---

In [ ]:
# 从零实现：两层网络的完整前向和反向传播

class TwoLayerNetwork:
    """
    两层全连接神经网络
    
    结构:
    x -> [W1, b1] -> z1 -> σ -> a1 -> [W2, b2] -> z2 -> σ -> a2 -> L
    """
    
    def __init__(self, input_size, hidden_size, output_size, activation='sigmoid'):
        """
        初始化网络参数
        
        参数:
        -----
        input_size : int
            输入维度 n0
        hidden_size : int
            隐藏层维度 n1
        output_size : int
            输出维度 n2
        activation : str
            激活函数类型 ('sigmoid', 'tanh', 'relu')
        """
        # 第1层参数（使用 Xavier 初始化）
        self.W1 = np.random.randn(hidden_size, input_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros(hidden_size)
        
        # 第2层参数
        self.W2 = np.random.randn(output_size, hidden_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros(output_size)
        
        # 激活函数选择
        self.activation = activation
        if activation == 'sigmoid':
            self.act_func = sigmoid
            self.act_deriv = sigmoid_derivative
        elif activation == 'tanh':
            self.act_func = tanh
            self.act_deriv = tanh_derivative
        elif activation == 'relu':
            self.act_func = relu
            self.act_deriv = relu_derivative
        else:
            raise ValueError(f"未知激活函数: {activation}")
        
        # 缓存变量（用于反向传播）
        self.cache = {}
    
    def forward(self, x):
        """
        前向传播
        
        参数:
        -----
        x : ndarray, shape (n0,)
            输入向量
        
        返回:
        -----
        a2 : ndarray, shape (n2,)
            输出向量
        """
        # 保存输入
        self.cache['x'] = x
        
        # 第1层前向传播
        # z1 = W1 @ x + b1
        z1 = np.dot(self.W1, x) + self.b1
        self.cache['z1'] = z1
        
        # a1 = σ(z1)
        a1 = self.act_func(z1)
        self.cache['a1'] = a1
        
        # 第2层前向传播
        # z2 = W2 @ a1 + b2
        z2 = np.dot(self.W2, a1) + self.b2
        self.cache['z2'] = z2
        
        # a2 = σ(z2)
        a2 = self.act_func(z2)
        self.cache['a2'] = a2
        
        return a2
    
    def backward(self, y):
        """
        反向传播：计算所有参数的梯度
        
        参数:
        -----
        y : ndarray, shape (n2,)
            目标输出
        
        返回:
        -----
        grads : dict
            包含所有参数的梯度
        """
        # 从缓存中获取前向传播的中间值
        x = self.cache['x']
        z1 = self.cache['z1']
        a1 = self.cache['a1']
        z2 = self.cache['z2']
        a2 = self.cache['a2']
        
        # ===== 第2层反向传播 =====
        
        # 输出层误差: δ2 = (a2 - y) ⊙ σ'(z2)
        delta2 = (a2 - y) * self.act_deriv(z2)
        
        # 第2层梯度
        grad_W2 = np.outer(delta2, a1)  # δ2 @ a1^T
        grad_b2 = delta2
        
        # ===== 第1层反向传播 =====
        
        # 隐藏层误差: δ1 = (W2^T @ δ2) ⊙ σ'(z1)
        # 步骤拆解:
        # 1. W2^T @ δ2: 将误差反向传播到隐藏层
        # 2. ⊙ σ'(z1): 乘以激活函数导数
        delta1 = np.dot(self.W2.T, delta2) * self.act_deriv(z1)
        
        # 第1层梯度
        grad_W1 = np.outer(delta1, x)  # δ1 @ x^T
        grad_b1 = delta1
        
        return {
            'W2': grad_W2,
            'b2': grad_b2,
            'W1': grad_W1,
            'b1': grad_b1
        }
    
    def compute_loss(self, y):
        """
        计算 MSE 损失
        
        L = 0.5 * ||a2 - y||²
        """
        a2 = self.cache['a2']
        return 0.5 * np.sum((a2 - y) ** 2)

# 测试两层网络
print("=" * 60)
print("两层神经网络前向+反向传播示例")
print("=" * 60)

np.random.seed(42)

# 创建网络: 3 -> 4 -> 2
net = TwoLayerNetwork(input_size=3, hidden_size=4, output_size=2, activation='sigmoid')

# 准备数据
x = np.array([1.0, 2.0, 3.0])
y = np.array([1.0, 0.0])

# 前向传播
a2 = net.forward(x)
loss = net.compute_loss(y)

print(f"\n网络结构: {x.shape[0]} -> {net.W1.shape[0]} -> {net.W2.shape[0]}")
print(f"激活函数: {net.activation}")
print(f"\n输入 x: {x}")
print(f"隐藏层激活 a1: {net.cache['a1']}")
print(f"输出 a2: {a2}")
print(f"目标 y: {y}")
print(f"损失 L: {loss:.6f}")

# 反向传播
grads = net.backward(y)

print("\n" + "="*60)
print("梯度计算结果")
print("="*60)
for name in ['W1', 'b1', 'W2', 'b2']:
    print(f"\n∂L/∂{name} (shape: {grads[name].shape}):")
    print(grads[name])

---

## 2.3 梯度消失与梯度爆炸

### 🔍 问题根源

在多层网络中，梯度需要逐层反向传播：

$$
\delta^{(l)} = \left( (W^{(l+1)})^T \delta^{(l+1)} \right) \odot \sigma'(\mathbf{z}^{(l)})
$$

对于 $L$ 层网络，第1层的梯度包含 $L-1$ 次矩阵乘法和激活函数导数：

$$
\frac{\partial L}{\partial W^{(1)}} \propto \prod_{l=2}^{L} (W^{(l)})^T \cdot \prod_{l=1}^{L-1} \sigma'(\mathbf{z}^{(l)})
$$

---

### ⚠️ 梯度消失（Vanishing Gradient）

**条件：**
- 激活函数导数 $\sigma'(z) < 1$（如 Sigmoid: $0 < \sigma'(z) \leq 0.25$）
- 权重较小

**结果：**

$$
\prod_{l=1}^{L-1} \sigma'(\mathbf{z}^{(l)}) \approx 0.25^{L-1} \to 0 \quad (L \to \infty)
$$

底层（靠近输入）的梯度趋近于0，无法学习。

---

### 💥 梯度爆炸（Exploding Gradient）

**条件：**
- 激活函数导数较大
- 权重初始化过大

**结果：**

$$
\prod \text{(权重 × 导数)} \to \infty
$$

梯度指数级增长，导致数值溢出。

---

### ✅ 解决方案

1. **使用 ReLU 激活函数**
   - $z > 0$ 时 $\sigma'(z) = 1$，不会衰减梯度

2. **批标准化（Batch Normalization）**
   - 稳定激活值的分布

3. **残差连接（Residual Connection）**
   - 提供梯度的"高速公路"：$\mathbf{a}^{(l+1)} = \mathbf{a}^{(l)} + F(\mathbf{a}^{(l)})$

4. **梯度裁剪（Gradient Clipping）**
   - 限制梯度范数：$\mathbf{g} \leftarrow \min\left(1, \frac{\text{threshold}}{\|\mathbf{g}\|}\right) \mathbf{g}$

---

In [ ]:
# 演示梯度消失问题

def demonstrate_vanishing_gradient(n_layers=10, activation='sigmoid'):
    """
    模拟深层网络的梯度消失
    
    参数:
    -----
    n_layers : int
        网络层数
    activation : str
        激活函数类型
    """
    print(f"\n{'='*60}")
    print(f"梯度消失演示：{n_layers}层网络，激活函数={activation}")
    print(f"{'='*60}\n")
    
    # 初始化参数
    layer_size = 10
    weights = []
    z_values = []
    
    # 随机初始化权重和激活值
    for i in range(n_layers):
        W = np.random.randn(layer_size, layer_size) * 0.1
        weights.append(W)
        
        # 模拟激活值（在激活函数的有效范围内）
        z = np.random.randn(layer_size) * 2
        z_values.append(z)
    
    # 选择激活函数
    if activation == 'sigmoid':
        act_deriv = sigmoid_derivative
    elif activation == 'tanh':
        act_deriv = tanh_derivative
    elif activation == 'relu':
        act_deriv = relu_derivative
    
    # 模拟反向传播：从输出层到输入层
    delta = np.ones(layer_size)  # 初始梯度
    gradient_norms = [np.linalg.norm(delta)]
    
    for l in range(n_layers - 1, 0, -1):
        # δ^(l-1) = (W^(l))^T @ δ^(l) ⊙ σ'(z^(l-1))
        delta = np.dot(weights[l].T, delta) * act_deriv(z_values[l-1])
        gradient_norms.append(np.linalg.norm(delta))
    
    # 反转列表（使其从输入层到输出层）
    gradient_norms = gradient_norms[::-1]
    
    # 打印结果
    print("各层梯度范数:")
    for i, norm in enumerate(gradient_norms):
        print(f"  第{i+1}层: {norm:.6e}")
    
    return gradient_norms

# 对比不同激活函数
np.random.seed(42)

sigmoid_grads = demonstrate_vanishing_gradient(n_layers=10, activation='sigmoid')
tanh_grads = demonstrate_vanishing_gradient(n_layers=10, activation='tanh')
relu_grads = demonstrate_vanishing_gradient(n_layers=10, activation='relu')

# 可视化
plt.figure(figsize=(12, 6))

layers = range(1, len(sigmoid_grads) + 1)
plt.plot(layers, sigmoid_grads, 'o-', linewidth=2, markersize=8, label='Sigmoid')
plt.plot(layers, tanh_grads, 's-', linewidth=2, markersize=8, label='Tanh')
plt.plot(layers, relu_grads, '^-', linewidth=2, markersize=8, label='ReLU')

plt.xlabel('层数（从输入到输出）', fontsize=14)
plt.ylabel('梯度范数', fontsize=14)
plt.title('不同激活函数的梯度范数变化', fontsize=16, fontweight='bold')
plt.yscale('log')  # 对数坐标
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n观察:")
print("1. Sigmoid: 梯度快速衰减，出现严重梯度消失")
print("2. Tanh: 比 Sigmoid 好，但仍有梯度消失")
print("3. ReLU: 梯度保持较稳定，有效缓解梯度消失")

---

# 3️⃣ 向量化批量实现

## 3.1 批量数据的前向传播

实际训练中使用 **mini-batch** 提高效率。

### 📊 形状约定（批量）

设 batch size = $B$：

$$
\begin{aligned}
X &\in \mathbb{R}^{B \times n_0} \quad &\text{(B个样本)} \\
Z^{(1)} &= XW^{(1)T} + b^{(1)} \in \mathbb{R}^{B \times n_1} \quad &\text{(广播偏置)} \\
A^{(1)} &= \sigma(Z^{(1)}) \in \mathbb{R}^{B \times n_1} \quad &\text{(逐元素)} \\
Z^{(2)} &= A^{(1)}W^{(2)T} + b^{(2)} \in \mathbb{R}^{B \times n_2} \\
A^{(2)} &= \sigma(Z^{(2)}) \in \mathbb{R}^{B \times n_2}
\end{aligned}
$$

---

## 3.2 批量数据的反向传播

梯度需要对 batch 维度求平均：

$$
\Delta^{(2)} = (A^{(2)} - Y) \odot \sigma'(Z^{(2)}) \quad \in \mathbb{R}^{B \times n_2}
$$

$$
\frac{\partial L}{\partial W^{(2)}} = \frac{1}{B} (\Delta^{(2)})^T A^{(1)} \quad \in \mathbb{R}^{n_2 \times n_1}
$$

$$
\frac{\partial L}{\partial b^{(2)}} = \frac{1}{B} \sum_{i=1}^{B} \delta^{(2)}_i = \frac{1}{B} \mathbf{1}^T \Delta^{(2)} \quad \in \mathbb{R}^{n_2}
$$

其中 $\mathbf{1} \in \mathbb{R}^B$ 是全1向量，对应 `np.sum(..., axis=0)`。

---

In [ ]:
# 从零实现：支持批量数据的两层网络

class TwoLayerNetworkBatch:
    """
    支持批量数据的两层神经网络
    
    关键区别:
    - 前向传播: X @ W^T (矩阵乘法)
    - 反向传播: 梯度需要对 batch 维度求平均
    """
    
    def __init__(self, input_size, hidden_size, output_size, activation='sigmoid'):
        # 参数初始化（与单样本版本相同）
        self.W1 = np.random.randn(hidden_size, input_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros(hidden_size)
        
        self.W2 = np.random.randn(output_size, hidden_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros(output_size)
        
        # 激活函数
        if activation == 'sigmoid':
            self.act_func = sigmoid
            self.act_deriv = sigmoid_derivative
        elif activation == 'relu':
            self.act_func = relu
            self.act_deriv = relu_derivative
        else:
            raise ValueError(f"未知激活函数: {activation}")
        
        self.cache = {}
    
    def forward(self, X):
        """
        批量前向传播
        
        参数:
        -----
        X : ndarray, shape (B, n0)
            批量输入
        
        返回:
        -----
        A2 : ndarray, shape (B, n2)
            批量输出
        """
        self.cache['X'] = X
        
        # 第1层
        # Z1 = X @ W1^T + b1
        # X: (B, n0), W1^T: (n0, n1) -> Z1: (B, n1)
        # b1: (n1,) 自动广播为 (B, n1)
        Z1 = np.dot(X, self.W1.T) + self.b1
        self.cache['Z1'] = Z1
        
        A1 = self.act_func(Z1)
        self.cache['A1'] = A1
        
        # 第2层
        Z2 = np.dot(A1, self.W2.T) + self.b2
        self.cache['Z2'] = Z2
        
        A2 = self.act_func(Z2)
        self.cache['A2'] = A2
        
        return A2
    
    def backward(self, Y):
        """
        批量反向传播
        
        参数:
        -----
        Y : ndarray, shape (B, n2)
            批量目标
        
        返回:
        -----
        grads : dict
            参数梯度（已求平均）
        """
        X = self.cache['X']
        Z1 = self.cache['Z1']
        A1 = self.cache['A1']
        Z2 = self.cache['Z2']
        A2 = self.cache['A2']
        
        B = X.shape[0]  # batch size
        
        # 第2层反向传播
        # Δ2 = (A2 - Y) ⊙ σ'(Z2)  shape: (B, n2)
        Delta2 = (A2 - Y) * self.act_deriv(Z2)
        
        # ∂L/∂W2 = (1/B) * Δ2^T @ A1
        # Δ2^T: (n2, B), A1: (B, n1) -> (n2, n1)
        grad_W2 = (1/B) * np.dot(Delta2.T, A1)
        
        # ∂L/∂b2 = (1/B) * sum(Δ2, axis=0)
        # 对 batch 维度求和  shape: (n2,)
        grad_b2 = (1/B) * np.sum(Delta2, axis=0)
        
        # 第1层反向传播
        # Δ1 = (Δ2 @ W2) ⊙ σ'(Z1)
        # Δ2: (B, n2), W2: (n2, n1) -> (B, n1)
        Delta1 = np.dot(Delta2, self.W2) * self.act_deriv(Z1)
        
        # ∂L/∂W1 = (1/B) * Δ1^T @ X
        grad_W1 = (1/B) * np.dot(Delta1.T, X)
        
        # ∂L/∂b1 = (1/B) * sum(Δ1, axis=0)
        grad_b1 = (1/B) * np.sum(Delta1, axis=0)
        
        return {
            'W2': grad_W2,
            'b2': grad_b2,
            'W1': grad_W1,
            'b1': grad_b1
        }
    
    def compute_loss(self, Y):
        """计算批量 MSE 损失"""
        A2 = self.cache['A2']
        return 0.5 * np.mean(np.sum((A2 - Y) ** 2, axis=1))

# 测试批量网络
print("=" * 60)
print("批量两层网络示例")
print("=" * 60)

np.random.seed(42)

# 创建网络
net_batch = TwoLayerNetworkBatch(input_size=3, hidden_size=4, output_size=2, activation='sigmoid')

# 批量数据 (batch_size=5)
X_batch = np.random.randn(5, 3)
Y_batch = np.random.randn(5, 2)

print(f"\n输入形状: {X_batch.shape}")
print(f"目标形状: {Y_batch.shape}")

# 前向传播
A2_batch = net_batch.forward(X_batch)
loss_batch = net_batch.compute_loss(Y_batch)

print(f"\n输出形状: {A2_batch.shape}")
print(f"损失: {loss_batch:.6f}")

# 反向传播
grads_batch = net_batch.backward(Y_batch)

print("\n" + "="*60)
print("梯度形状验证")
print("="*60)
print(f"W1: {net_batch.W1.shape} -> ∂L/∂W1: {grads_batch['W1'].shape} ✅")
print(f"b1: {net_batch.b1.shape} -> ∂L/∂b1: {grads_batch['b1'].shape} ✅")
print(f"W2: {net_batch.W2.shape} -> ∂L/∂W2: {grads_batch['W2'].shape} ✅")
print(f"b2: {net_batch.b2.shape} -> ∂L/∂b2: {grads_batch['b2'].shape} ✅")

---

## 3.3 完整训练示例：拟合非线性函数

使用两层网络拟合 $y = \sin(x)$，展示完整的训练流程。

In [ ]:
# 完整示例：训练两层网络拟合 sin(x)

print("=" * 60)
print("训练两层网络拟合 sin(x)")
print("=" * 60)

# 生成数据
np.random.seed(42)
n_samples = 200
X_train = np.linspace(-np.pi, np.pi, n_samples).reshape(-1, 1)  # (200, 1)
y_train = np.sin(X_train) + 0.1 * np.random.randn(n_samples, 1)  # 添加噪声

# 创建网络: 1 -> 10 -> 1
net = TwoLayerNetworkBatch(input_size=1, hidden_size=10, output_size=1, activation='sigmoid')

# 超参数
learning_rate = 0.5
n_epochs = 2000
batch_size = 32

# 训练记录
losses = []

print(f"\n数据集大小: {n_samples}")
print(f"网络结构: {net.W1.shape[1]} -> {net.W1.shape[0]} -> {net.W2.shape[0]}")
print(f"学习率: {learning_rate}")
print(f"训练轮数: {n_epochs}")
print(f"Batch size: {batch_size}")
print("\n开始训练...\n")

# 训练循环
for epoch in range(n_epochs):
    # 随机打乱数据
    indices = np.random.permutation(n_samples)
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]
    
    # Mini-batch 训练
    for i in range(0, n_samples, batch_size):
        X_batch = X_shuffled[i:i+batch_size]
        y_batch = y_shuffled[i:i+batch_size]
        
        # 前向传播
        net.forward(X_batch)
        
        # 反向传播
        grads = net.backward(y_batch)
        
        # 梯度下降更新参数
        net.W1 -= learning_rate * grads['W1']
        net.b1 -= learning_rate * grads['b1']
        net.W2 -= learning_rate * grads['W2']
        net.b2 -= learning_rate * grads['b2']
    
    # 计算整个训练集的损失
    net.forward(X_train)
    loss = net.compute_loss(y_train)
    losses.append(loss)
    
    if (epoch + 1) % 200 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}, Loss: {loss:.6f}")

print("\n训练完成！")
print(f"最终损失: {losses[-1]:.6f}")

In [ ]:
# 可视化训练结果

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左图：训练曲线
axes[0].plot(losses, linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=14)
axes[0].set_ylabel('Loss (MSE)', fontsize=14)
axes[0].set_title('训练曲线', fontsize=16, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')

# 右图：拟合结果
y_pred = net.forward(X_train)

axes[1].scatter(X_train, y_train, alpha=0.4, s=15, label='训练数据（带噪声）', color='gray')
axes[1].plot(X_train, y_pred, 'r-', linewidth=3, label='两层网络拟合')
axes[1].plot(X_train, np.sin(X_train), 'g--', linewidth=2, label='真实函数 sin(x)')
axes[1].set_xlabel('x', fontsize=14)
axes[1].set_ylabel('y', fontsize=14)
axes[1].set_title('拟合结果（非线性激活函数成功拟合）', fontsize=16, fontweight='bold')
axes[1].legend(fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n✅ 成功！两层网络通过反向传播学习到了非线性函数")
print("   这正是深度学习的核心能力：通过多层非线性变换逼近复杂函数")

---

# 🎓 小结与下一步

## ✅ 本教程你学到了什么

### 1. 单层网络反向传播
- 完整推导了梯度公式
- 理解了误差向量 $\delta$ 的含义
- 掌握了 $\frac{\partial L}{\partial W} = \delta \mathbf{x}^T$ 的核心公式
- 实现了常见激活函数的导数

### 2. 多层网络链式法则
- 误差的逐层反向传播机制
- 理解了梯度消失/爆炸的数学根源
- 对比了不同激活函数对梯度的影响
- 从零实现了完整的两层网络

### 3. 向量化批量实现
- 批量数据的形状处理技巧
- 广播机制在梯度计算中的应用
- 完整的训练循环实现
- 成功拟合非线性函数

---

## 🚀 下一步学习

现在你已经掌握了反向传播的数学原理，可以开始学习神经网络的实际应用：

- **neural_networks/01_perceptron_and_history.ipynb** - 神经网络的历史与感知器
- **neural_networks/02_mlp_and_forward_propagation.ipynb** - 多层感知器详解
- **neural_networks/03_backpropagation_algorithm.ipynb** - 反向传播算法的工程实现

---

## 📝 练习题

### 练习 1：三层网络
扩展 `TwoLayerNetworkBatch` 类，实现三层网络（2个隐藏层）。

### 练习 2：Softmax + 交叉熵
推导并实现 Softmax 激活函数和交叉熵损失的反向传播。

### 练习 3：梯度裁剪
在训练循环中实现梯度裁剪，观察对梯度爆炸的缓解效果。

### 练习 4：批标准化（挑战）
推导批标准化层的反向传播公式。

---

## 📚 参考资料

- [Backpropagation Algorithm - CS231n](http://cs231n.github.io/optimization-2/)
- [3Blue1Brown: 反向传播可视化](https://www.youtube.com/watch?v=Ilg3gGewQ5U)
- 《深度学习》Goodfellow - 第6章
- 《神经网络与深度学习》邱锡鹏 - 第3章

---

**恭喜你完成反向传播的数学推导！🎉**

你现在理解了深度学习训练的核心机制。